In [3]:
!cv2 version

/bin/bash: cv2: command not found


In [80]:
!pip install split-folders tqdm

In [92]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Generate batches of tensor image data with real-time data augmentation.
from tensorflow.keras.applications import MobileNetV2 # CNN architecture that works well w/mobile devices as well
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam # stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input # Preprocesses a tensor or Numpy array encoding a batch of images.
from tensorflow.keras.preprocessing.image import img_to_array, load_img 
from tensorflow.keras.utils import to_categorical # converts a class vector (integers) to a binary class matrix
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from imutils import paths
import cv2
import numpy as np
import pandas as pd
import os
import splitfolders
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16})
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.color'] = 'lightgrey'

## Preprocessing the data

In [93]:
# init the initial learning rate, number of epochs to train for, and batch size
init_lr = 1e-4
epochs = 20
bs = 32

In [94]:
DIRECTORY = r"data"
CATEGORIES = ["with_mask","without_mask","mask_weared_incorrect"]

In [95]:
print("[INFO] loading images...")

[INFO] loading images...


In [96]:
# get the list of images in our dataset directory, 
# then init the list of data and class images
# X = []
# y = []

# for category in CATEGORIES:
#     path = os.path.join(DIRECTORY, category) # path to mask, no_mask, incorrect
#     # looping over image paths and prepocessing the images
#     for img in os.listdir(path):
#         img_path = os.path.join(path,img)
#         image = load_img(img_path,target_size=(192,192)) # resize all images, target size
#         image = img_to_array(image)
#         image = preprocess_input(image)
#         X.append(image)
#         y.append(category)

In [97]:
# X = []
# y = []

# for category in CATEGORIES:
#     path = os.path.join(DIRECTORY, category)
#     for img in os.listdir(path):
#         image = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
#         image = img_to_array(image)
#         image = preprocess_input(image)
# #         plt.imshow(img_array, cmap='gray')
# #         plt.show()
#         X.append(img_array)
#         y.append(category)

In [98]:
# one-hot encoding on the labels (1 and 0) binary format
# mlb = LabelBinarizer()
# y = mlb.fit_transform(y)
# labels = to_categorical(labels)

In [99]:
# convert data to arrays b/c hidden layers only accept this format
# X = np.array(X,dtype='float32')
# y = np.array(y)

In [100]:
# split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
# splitfolders.ratio('data',output='split_data', seed=1337, ratio=(.8,.1,.1))

Copying files: 8982 files [00:01, 4933.32 files/s]


In [102]:
# construct the training image generator for data augmentation
# recreating the images

aug = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

In [103]:
train = aug.flow_from_directory('split_data/train', class_mode='categorical', batch_size=bs)
test = aug.flow_from_directory('split_data/test', class_mode='categorical',batch_size=bs)
val = aug.flow_from_directory('split_data/val',class_mode='categorical',batch_size=bs)

Found 7185 images belonging to 3 classes.
Found 900 images belonging to 3 classes.
Found 897 images belonging to 3 classes.


In [104]:
x_train, y_train = train.next()
x_test,y_test = test.next()
x_val, y_val = val.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (x_test.shape, x_test.min(), x_test.max()))

Batch shape=(32, 256, 256, 3), min=0.000, max=255.000


In [105]:
# load MobileNetV2 network, ensuring the head FC (Fully Connected) laery sets are left off
baseModel = MobileNetV2(input_shape=(192,192,3),weights="imagenet", include_top=False, input_tensor=Input(shape=(192,192,3)))

In [112]:
# create the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(6,6))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

In [113]:
# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# lopp over all layers in the base model and freeze them so they will not be updated during the first training process
for layer in baseModel.layers:
    layer.trainable=False

In [114]:
# compile model
print("[INFO] compiling model...")
opt = Adam(lr=init_lr,decay=init_lr / epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [115]:
# train head of the network
print("[INFO] training head...")
H = model.fit(
        train,
        steps_per_epoch=len(x_train) // bs,
        validation_data=val,
        validation_steps=len(x_val) //bs,
        epochs=epochs
)

[INFO] training head...
Epoch 1/20
1/1 [==============================] - 6s 6s/step - loss: 1.7500 - accuracy: 0.2812 - val_loss: 1.6487 - val_accuracy: 0.2188
Epoch 2/20
1/1 [==============================] - 2s 2s/step - loss: 1.5839 - accuracy: 0.3125 - val_loss: 1.5399 - val_accuracy: 0.2188
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 1.8017 - accuracy: 0.2188 - val_loss: 1.2866 - val_accuracy: 0.3750
Epoch 4/20
1/1 [==============================] - 2s 2s/step - loss: 1.0638 - accuracy: 0.5625 - val_loss: 1.3529 - val_accuracy: 0.1875
Epoch 5/20
1/1 [==============================] - 2s 2s/step - loss: 1.4288 - accuracy: 0.2812 - val_loss: 1.1485 - val_accuracy: 0.3750
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 1.4567 - accuracy: 0.3125 - val_loss: 1.2008 - val_accuracy: 0.2500
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 1.5568 - accuracy: 0.3438 - val_loss: 1.1030 - val_accuracy: 0.4375
Epoch 8/20
1/1 [=

KeyboardInterrupt: 

In [45]:
len(X_train),len(y_train)

(7185, 1797)